Connect to google drive

In [0]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


Video to frame

In [0]:
# %%shell
# # video resize
# for i in {1..85}
# {
#     ffmpeg -i $i.mp4 -vf scale=320:-1 0$i.mp4
# }

# # frame resize
# for i in {2..80}
# {
#   cd ../frames$i
#   for i in *.png
#   {
#     ffmpeg -i $i -vf scale=320:-1 r$i
#   }  
# }

In [0]:
# %%shell
# for i in {1..85}
# {
#     mkdir video$i
#     ffmpeg -i 0$i.mp4 video$i/%d.png
# }

Use openpose to generate frames (may take a long time)

In [0]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

In [0]:
# !cd openpose && ./build/examples/openpose/openpose.bin --video ../drive/My\ Drive/636/data/part5_data/f_pos/18.mp4 --face --display 0 --write_video ./output/018.mp4

In [0]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

In [0]:
# show_local_mp4_video('./output/026.mp4', width=320, height=280)

In [0]:
ls ../drive/My\ Drive/636/data/part5_data/makeup/

12.mp4  15.mp4


Generate facial landmarks input from frames

In [0]:
cd openpose

/content/openpose


In [0]:
# detect poses of images
%%shell
for i in {17..17}
{
   ./build/examples/openpose/openpose.bin --image_dir ../drive/My\ Drive/636/data/part5_data/$i --write_json ../drive/My\ Drive/636/data/json/$i --number_people_max 1 --keypoint_scale 3 --render_pose 0 --display 0 --face
}

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 55.152257 seconds.


In [0]:
# detect poses of videos
%%shell
for i in {20..21}
{
  # ./build/examples/openpose/openpose.bin --video ../$i.mp4 --write_video_fps 24 --write_video ../drive/My\ Drive/636/data/part5_data/makeup/0$i.mp4 --number_people_max 1 --keypoint_scale 3 --display 0 --face
  ./build/examples/openpose/openpose.bin --video ../drive/My\ Drive/636/data/part5_data/$i.mp4 --write_video_fps 24 --write_video ../drive/My\ Drive/636/data/part5_data/0$i.mp4 --write_json ../drive/My\ Drive/636/data/json/$i --number_people_max 1 --keypoint_scale 3 --display 0 --face
}

In [0]:
%%shell
cd '/content/drive/My Drive/636/data/json/0/'
ls -1 | grep -v ^l | wc -l

In [0]:
%%shell
cd '/content/drive/My Drive/636/data/json/1/'
for i in {1..27}
{
  cd ../$i
  echo $i && ls -1 | grep -v ^l | wc -l
}

1
15924
2
8983
3
17166
4
4249
5
19206
6
6105
7
3036
8
6644
9
7108
10
12905
11
3951
12
1458
13
23652
14
21125
15
7907
16
6223
17
0
18
300
19
180
20
393
21
288
22
32
23
121
24
71
25
95
26
100
27
288


JSON to CSV

In [0]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import json
import pandas as pd
import numpy as np
import os, os.path
import time
import csv

In [0]:
N_FACE = 70
N_POSE = 8
FOLDER = './drive/My Drive/636/data/json/'
CSV_FOLDER = './drive/My Drive/636/data/csv/'
LABEL_FOLDER = './drive/My Drive/636/data/label/'
COL_NAMES = ['vid', 'frame', 'label']
COL_NAMES += ['f_' + str(i) for i in range(N_FACE * 2)]
COL_NAMES += ['p_' + str(i) for i in range(N_POSE * 2)]

In [0]:
droplist_f = set([(i * 3 + 2) for i in range(N_FACE)])
droplist_p = set([(i * 3 + 2) for i in range(N_POSE)])
idx_f = [i for i in range(N_FACE * 3) if (i not in droplist_f)]
idx_p = [i for i in range(N_POSE * 3) if (i not in droplist_p)]

In [0]:
# collect the json output of openpose and combine them into dataframe
def json_to_df(folderpath, csv_path, label_path, raw_label):
  start_time = time.time()
  idx = 0
  row = 0
  matrix = []
  label = []
  for name in sorted(os.listdir(folderpath)): 
    if idx < 15000 and name[-4:] == 'json':
      filepath = os.path.join(folderpath, name)
      if os.path.isfile(filepath):
        with open(filepath, 'r') as f:
          data = json.load(f)['people']
          if len(data) > 0:
            label.append(raw_label[idx])
            matrix.append([])
            for j in range(N_FACE):
                matrix[row].append(float(data[0]['face_keypoints_2d'][j * 3]))
                matrix[row].append(float(data[0]['face_keypoints_2d'][j * 3 + 1]))
            for j in range(N_POSE):
              matrix[row].append(float(data[0]['pose_keypoints_2d'][j * 3]))
              matrix[row].append(float(data[0]['pose_keypoints_2d'][j * 3 + 1]))
            row += 1
      idx += 1
  end_time = time.time()
  print(end_time-start_time)
  df = pd.DataFrame(matrix, columns = COL_NAMES)
  df_label = pd.DataFrame(label, columns = ['label'])  
  print(df.shape, df_label.shape)
  # dataframe to csv and upload to google drive 
  df.to_csv(csv_path)
  df_label.to_csv(label_path)

In [0]:
def get_df(i, label):
  folderpath = FOLDER + str(i) + '/'
  csv_path = CSV_FOLDER + str(i) + '.csv'
  label_path = LABEL_FOLDER + str(i) + '.csv'
  json_to_df(folderpath, csv_path, label_path, label)

In [0]:
label_0 = [0 for i in range(2331)]
for i in range(29 * 30, 37 * 30):
  label_0[i] = 1
for i in range(50 * 30, 69 * 30):
  label_0[i] = 1
get_df(0, label_0)

0 0
500 500
1000 1000
1500 1500
2000 2000
(2331, 156) (2331, 1)


In [0]:
label_1 = [0 for i in range(15000)]
for i in range(30*296 - 240, 30*420 - 240):
  label_1[i] = 1
for i in range(30*440 - 240, 15000):
  label_1[i] = 1
get_df(1, label_1)

In [0]:
# collect the json output of openpose and combine them into csv
def json_to_csv(folderpath, csv_path, label_path, raw_label, vid):
  start_time = time.time()
  idx = 0
  write_file = open(csv_path, 'w')
  csv_writer = csv.writer(write_file)
  csv_writer.writerow(COL_NAMES) 
  for name in sorted(os.listdir(folderpath)): 
    if idx % 500 == 0:
      print(idx)
    if name[-4:] == 'json':
      filepath = os.path.join(folderpath, name)
      if os.path.isfile(filepath):
        with open(filepath, 'r') as f:
          data = json.load(f)['people']
          if len(data) > 0:
            row = [vid, name, raw_label[idx]]
            row_face = [float(data[0]['face_keypoints_2d'][j]) for j in idx_f]
            row_pose = [float(data[0]['pose_keypoints_2d'][j]) for j in idx_p]
            row += row_face + row_pose
            row = np.array(row)
            csv_writer.writerow(row)
    idx += 1
  end_time = time.time()
  print(end_time-start_time)

In [0]:
def get_csv(i, label):
  folderpath = FOLDER + str(i) + '/'
  csv_path = CSV_FOLDER + str(i) + '.csv'
  label_path = LABEL_FOLDER + str(i) + '.csv'
  json_to_csv(folderpath, csv_path, label_path, label, i)

In [0]:
label_2 = [0 for i in range(8983)]
for i in range(24*66, 24*112):
  label_2[i] = 1
for i in range(24*151, 24*201):
  label_2[i] = 1

In [132]:
get_csv(2, label_2)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
2899.540365457535


In [0]:
# collect the json output of openpose and combine them into csv
def json_to_csv(folderpath, csv_path, label_path, raw_label, vid):
  start_time = time.time()
  idx = 0
  write_file = open(csv_path, 'w')
  csv_writer = csv.writer(write_file)
  csv_writer.writerow(COL_NAMES) 
  for name in sorted(os.listdir(folderpath)): 
    if idx < 15000 and name[-4:] == 'json':
      if idx % 500 == 0:
        print(idx)
      filepath = os.path.join(folderpath, name)
      if os.path.isfile(filepath):
        with open(filepath, 'r') as f:
          data = json.load(f)['people']
          if len(data) > 0:
            row = [vid, name, raw_label[idx]]
            row_face = [float(data[0]['face_keypoints_2d'][j]) for j in idx_f]
            row_pose = [float(data[0]['pose_keypoints_2d'][j]) for j in idx_p]
            row += row_face + row_pose
            row = np.array(row)
            csv_writer.writerow(row)
      idx += 1
    elif idx >= 15000:
      break
  end_time = time.time()
  print(end_time-start_time)

In [134]:
label_5 = [0 for i in range(15000)]
for i in range(30*424, 30*464):
  label_5[i] = 1
for i in range(30*487, 15000):
  label_5[i] = 1
get_csv(5, label_5)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
4840.859538793564


In [0]:
# collect the json output of openpose and combine them into csv
def json_to_csv(folderpath, csv_path, raw_label, vid):
  start_time = time.time()
  idx = 0
  write_file = open(csv_path, 'w')
  csv_writer = csv.writer(write_file)
  csv_writer.writerow(COL_NAMES) 
  for name in sorted(os.listdir(folderpath)): 
    if name[-4:] == 'json':
      if idx % 500 == 0:
        print(idx)
      filepath = os.path.join(folderpath, name)
      if os.path.isfile(filepath):
        with open(filepath, 'r') as f:
          data = json.load(f)['people']
          if len(data) > 0:
            row = [vid, name, raw_label[idx]]
            row_face = [float(data[0]['face_keypoints_2d'][j]) for j in idx_f]
            row_pose = [float(data[0]['pose_keypoints_2d'][j]) for j in idx_p]
            row += row_face + row_pose
            row = np.array(row)
            csv_writer.writerow(row)
      idx += 1
  end_time = time.time()
  print(end_time-start_time)

In [0]:
def get_csv(i, label):
  folderpath = FOLDER + str(i) + '/'
  csv_path = CSV_FOLDER + str(i) + '.csv'
  json_to_csv(folderpath, csv_path, label, i)

In [138]:
label_10 = [0 for i in range(12905)]
for i in range(5669, 10819):
  label_10[i] = 1

get_csv(10, label_10)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
3937.282928943634


In [0]:
# collect the json output of openpose and combine them into csv
def json_to_csv(folderpath, csv_path, raw_label, vid):
  start_time = time.time()
  idx = 0
  write_file = open(csv_path, 'w')
  csv_writer = csv.writer(write_file)
  csv_writer.writerow(COL_NAMES) 
  for name in sorted(os.listdir(folderpath))[5000:15000]: 
    if name[-4:] == 'json':
      if idx % 500 == 0:
        print(idx)
      filepath = os.path.join(folderpath, name)
      if os.path.isfile(filepath):
        with open(filepath, 'r') as f:
          data = json.load(f)['people']
          if len(data) > 0:
            row = [vid, name, raw_label[idx]]
            row_face = [float(data[0]['face_keypoints_2d'][j]) for j in idx_f]
            row_pose = [float(data[0]['pose_keypoints_2d'][j]) for j in idx_p]
            row += row_face + row_pose
            row = np.array(row)
            csv_writer.writerow(row)
      idx += 1
  end_time = time.time()
  print(end_time-start_time)

In [140]:
label_13 = [0 for i in range(10000)]
for i in range(5669, 10000):
  label_13[i] = 1
get_csv(13, label_13)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
3064.4089555740356


In [0]:
# collect the json output of openpose and combine them into csv
def json_to_csv(folderpath, csv_path, raw_label, vid):
  start_time = time.time()
  idx = 0
  write_file = open(csv_path, 'w')
  csv_writer = csv.writer(write_file)
  csv_writer.writerow(COL_NAMES) 
  for name in sorted(os.listdir(folderpath)): 
    if name[-4:] == 'json':
      if idx % 500 == 0:
        print(idx)
      filepath = os.path.join(folderpath, name)
      if os.path.isfile(filepath):
        with open(filepath, 'r') as f:
          data = json.load(f)['people']
          if len(data) > 0:
            row = [vid, name, raw_label[idx]]
            row_face = [float(data[0]['face_keypoints_2d'][j]) for j in idx_f]
            row_pose = [float(data[0]['pose_keypoints_2d'][j]) for j in idx_p]
            row += row_face + row_pose
            row = np.array(row)
            csv_writer.writerow(row)
      idx += 1
  end_time = time.time()
  print(end_time-start_time)

In [142]:
label_16 = [0 for i in range(6223)]
for i in range(2670, 5250):
  label_16[i] = 1

get_csv(16, label_16)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
1956.5804691314697


In [0]:
# collect the json output of openpose and combine them into csv
def json_to_csv(folderpath, csv_path, raw_label, vid):
  start_time = time.time()
  idx = 0
  write_file = open(csv_path, 'w')
  csv_writer = csv.writer(write_file)
  csv_writer.writerow(COL_NAMES) 
  for name in sorted(os.listdir(folderpath))[:15000]: 
    if name[-4:] == 'json':
      if idx % 500 == 0:
        print(idx)
      filepath = os.path.join(folderpath, name)
      if os.path.isfile(filepath):
        with open(filepath, 'r') as f:
          data = json.load(f)['people']
          if len(data) > 0:
            row = [vid, name, raw_label[idx]]
            row_face = [float(data[0]['face_keypoints_2d'][j]) for j in idx_f]
            row_pose = [float(data[0]['pose_keypoints_2d'][j]) for j in idx_p]
            row += row_face + row_pose
            row = np.array(row)
            csv_writer.writerow(row)
      idx += 1
  end_time = time.time()
  print(end_time-start_time)

In [144]:
label_1 = [0 for i in range(15000)]
for i in range(30*296 - 240, 30*420 - 240):
  label_1[i] = 1
for i in range(30*440 - 240, 15000):
  label_1[i] = 1
get_csv(1, label_1)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
34.529720067977905


Reshape Data into Segments and Prepare for Keras

In [0]:
# slice every 10 frame as a segment until the last frame reaches the end
def get_segments_and_timestamps(df, fps, STEP):
  slices = []
  ts = []
  size = df.shape[0]
  i = STEP
  if size >= STEP:
    while i <= size:
      if i >= STEP:
        slices.append(df.values[df.index[i - STEP : i]])
        ts.append(i/fps)
      i += 1
  else:
    slice_ = df.values
    fillzero = np.zeros((STEP - size, N_FEATURES))
    slice_ = np.concatenate((slice_, fillzero), axis=0)
    slices.append(slice_)
  slices_arr = np.asarray(slices, dtype= np.float32)
  ts_arr = np.asarray(ts, dtype= np.float32)
  return slices_arr, ts_arr